In [24]:
import json
import pandas as pd
import requests
from tqdm import tqdm
from datetime import datetime
from collections import defaultdict 

In [25]:
# ETAPE 1 connexion à l'API + récupération offres

def create_url(start_url="http://api.adzuna.com/v1/api/jobs/"):
    """
    concactène l'adresse url à partir des paramètres choisis
    comment améliorer pour que les paramètres soient correctement passés ?
    """
    url_list = []
    for page_number in range(1, 100):
        #start_url = "http://api.adzuna.com/v1/api/jobs/"
        country = "fr"
        part0_url = "/search/"
        number_page = str(page_number) 
        part1_url = "?app_id="
        api_id = "37fe08af"
        part2_url = "&app_key="
        api_key = "17c1369ea8fb68bc48f834aebc0bec53"
        part3_url = "&results_per_page="
        results_per_page = str(25)
        part4_url = "&what_and="
        what_and = "data%20engineer"
        part5_url = "&max_days_old="
        max_days_old = str(30)
        end_url = "&content-type=application/json"
        full_url = f"{start_url}{country}{part0_url}{number_page}{part1_url}{api_id}{part2_url}{api_key}{part3_url}{results_per_page}{part4_url}{what_and}{part5_url}{max_days_old}{end_url}"
        url_list.append(full_url)
    return url_list

In [26]:
def scrape_urls(url):
    """
    lance la requête sur l'api Adzuna pour chacunes des url crééés
    si le status_code est incorrect, il s'affiche et l'url liée s'affiche
    """
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("The request for {} was not successful : status {}".format(url, response.status_code))
        return None

In [27]:
class Company:
    def __init__(self):
        self.name = None
        self.sector = None
        self.employees = None
        self.creation_year = None
        self.turnover = None
        self.mean_age = None


class JobOffer:
    def __init__(self):
        self.title = None
        self.company = None
        self.contract_type = None
        self.location = None
        self.salary = None
        self.remote_type = None
        self.starting_date = None
        self.required_experience = None
        self.education = None
        self.description = None
        self.profil_experience = None
        self.publication_date = None
        self.url_direct_offer = None
        

class GetJobOffer():
    def __init__(self, full_info):
        self.full_info = full_info
        self.job_offer = JobOffer()
        self.company = Company()
        

    def get_job_details(self):
        results = self.full_info['results'][0]
        self.job_offer.title = results["title"] if "title" in results else None 
        self.job_offer.id_source = results["id"] if "id" in results else None
        self.job_offer.location = results["location"]["display_name"] if "location" in results else None
        self.job_offer.company = vars(self.company)
        if ("contract_type" in results) or ("contract_time" in results):
            if ("contract_type" in results) and ("contract_time" in results):
                self.job_offer.contract_type = results["contract_type"] + " and " + results["contract_time"]
            elif ("contract_type" in results) and ("contract_time" not in results):
                self.job_offer.contract_type = results["contract_type"] 
            else : 
                self.job_offer.contract_type = results["contract_time"] 
        self.job_offer.salary = results["salary_is_predicted"] if "salary_is_predicted" in results else None
        self.job_offer.description = results["description"] if "description" in results else None
        self.job_offer.publication_date = results["created"][:10] if "created" in results else None
        self.job_offer.url_direct_offer = results["redirect_url"] if "redirect_url" in results else None
        
    
    def get_company_details(self):
        results = self.full_info['results'][0]
        if "category" in results:
            self.company.sector = self.full_info['results'][0]["category"]["tag"]
        if 'company' in results:
            if "display_name" in self.full_info['results'][0]['company']:
                self.company.name = self.full_info['results'][0]['company']["display_name"]
            elif "name" in self.full_info['results'][0]['company']:
                self.company.name = self.full_info['results'][0]['company']["name"]


    def combine_job_company(self):
        self.get_company_details()
        self.get_job_details()
        return vars(self.job_offer)


In [28]:
def final_step(jobs_offers):
    jobs_ready_for_export = []
    for offer in jobs_offers:
        getjoboffer = GetJobOffer(offer)
        jobs_ready_for_export.append(getjoboffer.combine_job_company())
    return jobs_ready_for_export    

In [29]:
url_list = create_url()
print(url_list[5])
jobs_offers = [scrape_urls(url) for url in tqdm(url_list, desc="Accessing API")]
jobs_ready_for_export = final_step(jobs_offers)

df_adzuna_jobs = pd.DataFrame(jobs_ready_for_export)

df_adzuna_jobs.to_csv('output/job_offers_wttj.csv')
df_adzuna_jobs

http://api.adzuna.com/v1/api/jobs/fr/search/6?app_id=37fe08af&app_key=17c1369ea8fb68bc48f834aebc0bec53&results_per_page=25&what_and=data%20engineer&max_days_old=30&content-type=application/json


Accessing API: 100%|█| 99/99 [02:48<00:00,  1.70s/


,title,company,contract_type,location,salary,remote_type,starting_date,required_experience,education,description,profil_experience,publication_date,url_direct_offer,id_source
0,Data Engineer PySpark,"{'name': 'THALES', 'sector': 'unknown', 'emplo...",permanent and full_time,"Toulouse, Haute-Garonne",0,None,None,None,None,QUI SOMMES-NOUS ? Thales propose des systèmes ...,None,2024-02-19,https://www.adzuna.fr/land/ad/4574739084?se=mt...,4574739084
1,Data Engineer Talend H/F,"{'name': 'Michael Page', 'sector': 'unknown', ...",permanent and full_time,France,0,None,None,None,None,Un acteur majeur dans les sciences de la vie e...,None,2024-02-09,https://www.adzuna.fr/land/ad/4563521532?se=sM...,4563521532
2,Data Engineer - Financial Services F/H,"{'name': 'CGI France', 'sector': 'unknown', 'e...",permanent and full_time,"Paris, Ile-de-France",0,None,None,None,None,"Position Description: Vous rejoignez CGI, lead...",None,2024-02-09,https://www.adzuna.fr/land/ad/4563520917?se=Ti...,4563520917
3,Data Engineer Senior,"{'name': 'AXA en France', 'sector': 'legal-job...",None,"Hauts-de-Seine, Ile-de-France",0,None,None,None,None,Environnement En tant que Senior Data Engineer...,None,2024-01-26,https://www.adzuna.fr/land/ad/4544977054?se=DF...,4544977054
4,Data Engineer F/H,"{'name': 'CGI', 'sector': 'unknown', 'employee...",None,"Niort, Deux-Sèvres",0,None,None,None,None,"Big Data, Data Science, Data analyse, Data arc...",None,2024-02-10,https://www.adzuna.fr/land/ad/4564417075?se=3n...,4564417075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Ingénieur DevOps F/H,"{'name': 'ATOS FRANCE', 'sector': 'unknown', '...",permanent and full_time,"Lyon, Rhône",0,None,None,None,None,Qui nous sommes. Nous sommes une équipe d'expe...,None,2024-02-13,https://www.adzuna.fr/land/ad/4566661660?se=To...,4566661660
95,Internship - Geophysicist,"{'name': 'CGG', 'sector': 'unknown', 'employee...",None,"Massy, Palaiseau",0,None,None,None,None,"Job Description This role is for physicists, m...",None,2024-02-06,https://www.adzuna.fr/details/4558929904?utm_m...,4558929904
96,Business Developer H/F,"{'name': 'HORUS CONSEILS', 'sector': 'it-jobs'...",full_time,"Rezé, Nantes",0,None,None,None,None,Votre mission principale en tant que Business ...,None,2024-02-16,https://www.adzuna.fr/details/4570880154?utm_m...,4570880154
97,Job in Germany: Engineer Strategic Network Arc...,"{'name': 'NetCom BW GmbH', 'sector': 'unknown'...",None,France,0,None,None,None,None,General information Engineer Strategic Network...,None,2024-02-08,https://www.adzuna.fr/details/4562268043?utm_m...,4562268043


In [20]:
jobs_ready_for_export[-1]

{'title': 'Data Engineer F/H',
 'company': {'name': 'CGI France',
  'sector': 'unknown',
  'employees': None,
  'creation_year': None,
  'turnover': None,
  'mean_age': None,
  'location': 'CGI France'},
 'contract_type': 'permanent and full_time',
 'location': 'Tours, Indre-et-Loire',
 'salary': '0',
 'remote_type': None,
 'starting_date': None,
 'required_experience': None,
 'education': None,
 'description': "Position Description: Big Data, Data Science, Data analyse, Data architecture,  Ça n'a pas de secret pour vous ? Que vous commenciez votre carrière professionnelle ou que vous soyez spécialiste de l'une de ces disciplines, intégrer notre communauté Data, c'est l'assurance de progresser, innover, partager, vous certifier et rendre service à nos clients. Rejoignez notre centre d'excellence en innovation à Tours et rendez unique l'expérience digitale de nos clients en travaillant sur des sujets te…",
 'profil_experience': None,
 'publication_date': '2024-02-09',
 'url_direct_offer

In [21]:
df_adzuna_jobs.describe()

,title,company,contract_type,location,salary,remote_type,starting_date,required_experience,education,description,profil_experience,publication_date,url_direct_offer,id_source
count,4,4,4,4,4,0,0,0,0,4,0,4,4,4
unique,3,2,1,4,1,0,0,0,0,4,0,2,4,4
top,Data Engineer F/H,"{'name': 'CGI France', 'sector': 'unknown', 'e...",permanent and full_time,"Toulouse, Haute-Garonne",0,NaN,NaN,NaN,NaN,QUI SOMMES-NOUS ? Thales propose des systèmes ...,NaN,2024-02-09,https://www.adzuna.fr/land/ad/4574739084?se=uD...,4574739084
freq,2,3,4,1,4,NaN,NaN,NaN,NaN,1,NaN,3,1,1


La variable *jobs_offers* est un objet json donc type dictionnaire
on instancie *full_info* de la class GetJobOffer avec l'intégralité de *jobs_offers*

Les informations clés liés au poste se trouve dans *jobs_offers[results]* qui est une liste d'un seul élément, un dictionnaire dont chaque clé est une information utile 
- 'location' (elle même un dictionnaire)
    exemple 'location': {'__CLASS__': 'Adzuna::API::Response::Location', 'display_name': 'Toulouse, Haute-Garonne', 'area': ['France', 'Occitanie', 'Haute-Garonne', 'Toulouse']},
- 'category' (elle même un dictionnaire)
    exemple {'label': 'Unknown', '__CLASS__': 'Adzuna::API::Response::Category', 'tag': 'unknown'}
- title': 'Data Engineer PySpark'
- 'redirect_url': 'https://www.adzuna.fr/land/ad/4574739084?se=DGYosEfS7hGqg0-VTaQgFg&utm_medium=api&utm_source=37fe08af&v=FE7ECBBD26503585482CAD5270E4346D7E31A2D2'
- 'contract_time': 'full_time'
- 'contract_type': 'permanent'
- 'id': '4574739084'
- 'created': '2024-02-19T20:01:48Z'
- 'description'
    "QUI SOMMES-NOUS ? Thales propose des systèmes d'information et de communication sécurisés et interopérables pour les forces armées, les forces de sécurité et les opérateurs d'importance vitale. Ces activités, qui regroupent radiocommunications, réseaux, systèmes de protection, systèmes d'information critiques et cybersécurité, répondent aux besoins de marchés où l'utilisation des nouvelles technologies numériques est déterminante. Thales intervient tout au long de la chaîne de valeur, des équip…"
- 'salary_is_predicted': '0'
- 'company'(elle même un dictionnaire)
      {'display_name': 'THALES', '__CLASS__': 'Adzuna::API::Response::Company'}